# Modeling and solving search problems

Consider the following river crossing puzzle:

## Jealous Husbands Problem with an Island

There are n couples: $(𝐻_1,𝑊_1)$, $(𝐻_2,𝑊_2)$, …, $(𝐻_𝑛,𝑊_𝑛)$  where $𝐻_𝑖$ is husband $𝑖$ and $𝑊_𝑖$ is wife $𝑖$.

They need to cross from the left bank of a river to the right bank.

There is a boat that can carry at most two people at a time.

Important: There is an island midway between the left and right banks. The boat can stop at the island.

Rules:

1. Jealousy constraint: No wife may ever be in the presence of another husband (on any shore or the island or in the boat) unless her own husband is also present. Otherwise, fights (and failures) occur.

2. The boat needs at least one person to operate (no remote-controlled trips).

3. The boat can travel between:

            Left bank ↔ Island

            Island ↔ Right bank

4. All people, including the boat, start at the left bank.

Goal: get everyone (husbands and wives) safely to the right bank, obeying all the rules.


## Tasks:
1. Put the problem in a form of a search tree: root, nodes, solution and rules for constructing the children.
2. Solve it using a classic DFS
3. Solve it using a classic Back-tracking.
4. Solve it using the A* method.  

In [55]:
# husbands - H[]
# wives - W[]

# left bank - 0
# island - 1
# right bank - 2
# initial state - [ 0000...0, 0000...0, 0] ([ W, H, b])
# final state - [ 2222...2, 2222...2, 2 ] ([ W, H, b])
# boat - 0/1/2

In [56]:
from typing import List, Optional, Set

In [57]:
def is_valid_state(state: List[List[int]], n: int) -> bool:
    W, H = state[0], state[1]
    
    for location in range(3):
        wives_at_location = [i for i in range(n) if W[i] == location]
        husbands_at_location = [i for i in range(n) if H[i] == location]
        
        for wife_idx in wives_at_location:
            if wife_idx not in husbands_at_location:
                for husband_idx in husbands_at_location:
                    if husband_idx != wife_idx:
                        return False
    
    return True

In [58]:
def state_to_string(state: List[List[int]]) -> str:
    return str(state)

In [81]:
def generate_next_states(current_state: List[List[int]], num_couples: int) -> List[List[List[int]]]:
    wives_locations = current_state[0]
    husbands_locations = current_state[1]
    boat_location = current_state[2]
    
    valid_next_states = []
    
    travelers = []
    for couple_id in range(num_couples):
        if wives_locations[couple_id] == boat_location:
            travelers.append(('Wife', couple_id))
        if husbands_locations[couple_id] == boat_location:
            travelers.append(('Husband', couple_id))
    
    possible_destinations = []
    if boat_location == 0:
        possible_destinations = [1]
    elif boat_location == 1:
        possible_destinations = [0, 2]
    else:
        possible_destinations = [1]
    
    for destination in possible_destinations:
        for traveler_type, couple_id in travelers:
            new_wives_locations = wives_locations.copy()
            new_husbands_locations = husbands_locations.copy()
            
            if traveler_type == 'Wife':
                new_wives_locations[couple_id] = destination
            else:
                new_husbands_locations[couple_id] = destination
                
            new_state = [new_wives_locations, new_husbands_locations, destination]
            
            if is_valid_state(new_state, num_couples):
                valid_next_states.append(new_state)
        
        for i, (type1, id1) in enumerate(travelers):
            for j, (type2, id2) in enumerate(travelers[i+1:], i+1):
                new_wives_locations = wives_locations.copy()
                new_husbands_locations = husbands_locations.copy()
                
                if type1 == 'Wife':
                    new_wives_locations[id1] = destination
                else:
                    new_husbands_locations[id1] = destination
                
                if type2 == 'Wife':
                    new_wives_locations[id2] = destination
                else:
                    new_husbands_locations[id2] = destination
                
                new_state = [new_wives_locations, new_husbands_locations, destination]
                
                if is_valid_state(new_state, num_couples):
                    valid_next_states.append(new_state)
    
    return valid_next_states

In [60]:
def is_goal_state(state: List[List[int]], n: int) -> bool:
    W, H, boat = state[0], state[1], state[2]
    return all(w == 2 for w in W) and all(h == 2 for h in H) and boat == 2

In [ ]:
def dfs(initial_state: List[List[int]], n: int) -> List[List[List[int]]]:
    stack = [(initial_state, [initial_state])]
    
    visited = set([state_to_string(initial_state)])
    
    while stack:
        current_state, path = stack.pop()
        
        if is_goal_state(current_state, n):
            return path
        
        next_states = generate_next_states(current_state, n)
        
        for next_state in next_states:
            state_str = state_to_string(next_state)
            
            if state_str not in visited:
                visited.add(state_str)c 
                stack.append((next_state, path + [next_state]))
    
    return None

In [83]:
def backtracking(current_state: List[List[int]], n: int, visited: Set[str] = None, 
                     path: List[List[List[int]]] = None, depth: int = 0, max_depth: int = 30) -> Optional[List[List[List[int]]]]:
    if visited is None:
        visited = set()
    
    if path is None:
        path = [current_state]
    
    if is_goal_state(current_state, n):
        return path
    
    if depth >= max_depth:
        return None
    
    state_str = state_to_string(current_state)
    visited.add(state_str)
    
    next_states = generate_next_states(current_state, n)
    
    for next_state in reversed(next_states):
        next_state_str = state_to_string(next_state)
        
        if next_state_str not in visited:
            result = backtracking(next_state, n, visited, path + [next_state], depth + 1, max_depth)
            
            if result:
                return result
    
    return None

In [64]:
def heuristic(state: List[List[int]], n: int) -> int:
    W, H, boat = state[0], state[1], state[2]
    not_at_goal = sum(1 for w in W if w != 2) + sum(1 for h in H if h != 2)
    boat_penalty = 2 - boat
    return not_at_goal + boat_penalty

In [95]:
import heapq

def astar(initial_state: List[List[int]], num_couples: int) -> List[List[List[int]]]:
    priority_queue = [(heuristic(initial_state, num_couples), 0, id(initial_state), initial_state, [initial_state])]
    
    cost_to_reach = {state_to_string(initial_state): 0}
    
    explored_states = set([state_to_string(initial_state)])
    
    while priority_queue:
        _, moves_so_far, _, current_state, current_path = heapq.heappop(priority_queue)
        
        if is_goal_state(current_state, num_couples):
            return current_path
        
        candidate_states = generate_next_states(current_state, num_couples)
        
        for candidate_state in candidate_states:
            state_key = state_to_string(candidate_state)
            candidate_moves = moves_so_far + 1
            
            if state_key not in explored_states or candidate_moves < cost_to_reach.get(state_key, float('inf')):
                cost_to_reach[state_key] = candidate_moves
                
                estimated_remaining = heuristic(candidate_state, num_couples)
                estimated_total = candidate_moves + estimated_remaining
                
                heapq.heappush(
                    priority_queue, 
                    (estimated_total, candidate_moves, id(candidate_state), candidate_state, current_path + [candidate_state])
                )
                
                explored_states.add(state_key)
    
    return None

In [85]:
def print_solution(solution, n):
    if not solution:
        print("No solution found!")
        return
    locations = ["Left Bank", "Island", "Right Bank"]
    for step, state in enumerate(solution):
        W, H, boat = state[0], state[1], state[2]
        print(f"Step {step}:")
        print(f"Boat at: {locations[boat]}")
        for loc in range(3):
            people_at_loc = []
            for i in range(n):
                if W[i] == loc:
                    people_at_loc.append(f"W{i+1}")
                if H[i] == loc:
                    people_at_loc.append(f"H{i+1}")
            print(f"{locations[loc]}: {', '.join(people_at_loc) if people_at_loc else 'Empty'}")
        print()

In [94]:
def solve_jealous_husbands(n):
    initial_W = [0] * n
    initial_H = [0] * n
    initial_boat = 0
    
    initial_state = [initial_W, initial_H, initial_boat]
    
    solution = dfs(initial_state, n)
    print(f"DFS: {len(solution) - 1}")
    # print_solution(solution, n)

    solution = backtracking(initial_state, n, max_depth=50)
    print(f"Backtracking: {len(solution) - 1}")
    # print_solution(solution, n)

    solution = astar(initial_state, n)
    print(f"A*: {len(solution) - 1}")

    return solution

In [100]:
if __name__ == "__main__":
    number_of_couples = 3
    solution = solve_jealous_husbands(number_of_couples)

DFS: 94
Backtracking: 34
A*: 18
